In [1]:
import os

from tensorflow.python.keras.utils.np_utils import to_categorical

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import matplotlib.pyplot as plt
print(tf.__version__)

2.8.0


In [2]:
"""Initialization of data generator"""
from data_generator import DataGenerator
generator = DataGenerator(test_data=100, train_data=300)

In [3]:
"""Get train data"""
generator.generate_train_data()
train_data = generator.get_train_data()

In [4]:
"""Get test data"""
generator.generate_test_data()
test_data = generator.get_test_data()

In [9]:
"""Get shapes and examples of generated data"""
print(f'Train: {len(train_data)}, Test: {len(test_data)}')
print(f'Train ex.: {train_data[13]}')
print(f'Test ex.: {test_data[50]}')

Train: 900, Test: 100
Train ex.: RepresentedData(number=3, data=[1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])
Test ex.: RepresentedData(number=0, data=[0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0])


In [13]:
import random
n1 = random.randint(0, len(train_data) - 1)
print(n1)

405


In [14]:
# переделываем целевые значения
# y_train[i] -- это цифра, подлежащая различению, например 3
# Превратим её в массив [0 0 0 1 0 0 0 0 0 0]
# здесь 1 стоит на 3 месте. Это и есть целевые выходы
# нейронов последнего слоя, чтобы именно нейрон с номером 3
# откликался на изображение цифры 3
# Для такого преобразования есть специальная функция  to_categorical
# Такой формат называется вектор One-hot
print("Целевое значение до преобразования:", train_data[n1].number)

Целевое значение до преобразования: 5


In [15]:
train_data[n1].number = to_categorical(train_data[n1].number, 10)

In [16]:
print("Целевое значение после преобразования:", train_data[n1].number)


Целевое значение после преобразования: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [18]:
# Создаём модель слоя
class DenseNN(tf.Module):
    # Конструктор. Заполняет начальные значения
    def __init__(self, outputs, activate="relu"):
        super().__init__()
        self.outputs = outputs  # количество выходов = количеству нейронов в слое
        self.activate = activate  # тип активационной функции (по умолчанию relu)
        self.fl_init = False  # становится true после первой инициализации весов

    # Функция для расчёта выходных величин сети
    def __call__(self, x):
        if not self.fl_init:  # если весов ещё нет, создаём их случайными
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name="w")
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name="b")
            """Размер тензора w: (x.shape[-1], self.outputs)
            здесь x.shape[-1] - размер вектора входных данных,
            то есть, сколько входов в первый раз подадим на сеть,
            столько весов к каждому нейрону и сформируется
            следовательно, первый индекс - номер входа нейрона
            второй размер self.outputs совпадает с количеством выходов
            и с количеством нейронов в слое, значит второй
            индекс - номер нейрона
            stddev=0.1 - среднеквадратическое отклонение"""

            # преобразуем w и b в тензоры
            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)

            self.fl_init = True
        # матричное вычисление выхода слоя
        y = x @ self.w + self.b

        if self.activate == "relu":
            return tf.nn.relu(y)
        elif self.activate == "softmax":
            return tf.nn.softmax(y)

        return y


In [19]:
# создадим два полносвязных слоя со 128 нейронами и 10
layer_1 = DenseNN(128)
layer_2 = DenseNN(10, activate="softmax")


In [20]:
# Функция расчёта значений на выходе сети
def model_predict(x):
    y = layer_1(x)
    y = layer_2(y)
    return y  # layer_2(layer_1(x))


In [21]:
# Задание функции потерь
cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))
# y_true, y_pred – это наборы one-hot векторов размером мини-батча


In [22]:
# определим оптимизатор для градиентного спуска
opt = tf.optimizers.Adam(learning_rate=0.001)

In [23]:
# Готовим обучение
BATCH_SIZE = 32
EPOCHS = 10
TOTAL = len(train_data)
print(f'Data amount in training set: {TOTAL}')

Data amount in training set: 900


In [24]:
# ОСТАЛОСЬ РЕФАКТОРИТЬ ПОСЛЕ ЭТОГО

In [ ]:
# tf.data.Dataset - это класс с набором обучающих данных.
# Его можно создать из списков питона методом from_tensor_slices
# можно из файлов dataset = tf.data.TextLineDataset(["file1.txt", "file2.txt"])
# подробнее https://www.tensorflow.org/api_docs/python/tf/data/Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Перемешиваем выборки и разбиваем на батчи
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

In [ ]:
# Цикл обучения
for n in range(EPOCHS):
    loss = 0
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            f_loss = cross_entropy(y_batch, model_predict(x_batch))

        loss += f_loss
        grads = tape.gradient(f_loss, [layer_1.trainable_variables, layer_2.trainable_variables])
        opt.apply_gradients(zip(grads[0], layer_1.trainable_variables))
        opt.apply_gradients(zip(grads[1], layer_2.trainable_variables))

    print(f'Loss by epochs: {loss.numpy()}')

In [ ]:
y = model_predict(x_test)
y2 = tf.argmax(y, axis=1).numpy()
acc = len(y_test[y_test == y2]) / y_test.shape[0] * 100
print(f'Summary accuracy: {acc}')